### main search of Sign language video for each word in the sentence 


In [1]:
# https://dic.ttrs.or.th/video/view/62d920d866b04b724ea396dd 


# these videos cant be downloaded

In [3]:
import socket
import re

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

SVC_ACC = !(gcloud config get-value core/account)
SVC_ACC = SVC_ACC[0]

PROJECT_NUMBER=str(re.search(r'\d+', SVC_ACC).group())

LOCATION="us-central1"

UNIQUE_PREFIX = socket.gethostname()
UNIQUE_PREFIX = re.sub('[^A-Za-z0-9]+', '', UNIQUE_PREFIX)

UID = UNIQUE_PREFIX

BUCKET_NAME = f"{PROJECT_ID}-{UNIQUE_PREFIX}-{LOCATION}"

BUCKET_URI = f"gs://{BUCKET_NAME}"

! gcloud config set project $PROJECT_ID
! gcloud storage buckets create {BUCKET_URI} --project={PROJECT_ID} --location={LOCATION}

!mkdir content
!rm -r content/clips/
!mkdir content/clips/
!rm -r content/frames/
!mkdir content/frames/

Updated property [core/project].
Creating gs://my-project-0004-346516-pytorch112kagglewbi-us-central1/...
ERROR: (gcloud.storage.buckets.create) HTTPError 409: Your previous request to create the named bucket succeeded and you already own it.
mkdir: cannot create directory ‘content’: File exists


In [4]:
BUCKET_URI

'gs://my-project-0004-346516-pytorch112kagglewbi-us-central1'

In [5]:
import os
from moviepy.editor import VideoFileClip
from google.cloud import aiplatform
import pandas as pd
from datetime import datetime

# Project Setup (Replace placeholders)
BUCKET_URI = BUCKET_URI
BUCKET_URI_ME = f"{BUCKET_URI}/embeddings/"
LOCATION = 'us-central1'
PROJECT_ID =PROJECT_ID

aiplatform.init(project=PROJECT_ID, location=LOCATION)

# Video Folder Path
video_folder = "./videos" 
i =1
# Data Collection
shots_data = []
for filename in os.listdir(video_folder):
    if filename.endswith(".mp4"):
        video_path = os.path.join(video_folder, filename)
        clip = VideoFileClip(video_path)

        # Frame Extraction (Adjust as needed)
        frame = clip.get_frame(0)  

        shots_data.append({
            'id': f"{filename}_{i}",  # Unique ID
            'start_time': 0, 
            'end_time': clip.duration,  # Duration in seconds
            'clip_name': filename,
            'frame_name': f"{filename}_{i}.jpg", 
            'description': os.path.splitext(filename)[0], # Filename as description
            'embedding': "test_embedding"
        })

# DataFrame Creation
shots_df = pd.DataFrame(shots_data)



In [6]:
shots_df.head()

,id,start_time,end_time,clip_name,frame_name,description,embedding
0,be_careful_bk.mp4_1,0,2.34,be_careful_bk.mp4,be_careful_bk.mp4_1.jpg,be_careful_bk,test_embedding
1,sailor_bk.mp4_1,0,2.90,sailor_bk.mp4,sailor_bk.mp4_1.jpg,sailor_bk,test_embedding
2,should.mp4_1,0,4.74,should.mp4,should.mp4_1.jpg,should,test_embedding
3,today.mp4_1,0,2.16,today.mp4,today.mp4_1.jpg,today,test_embedding
4,afternoon.mp4_1,0,1.84,afternoon.mp4,afternoon.mp4_1.jpg,afternoon,test_embedding


In [7]:
from vertexai.language_models import TextEmbeddingModel
text_embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")

In [8]:
embeddingsList = []
for description in shots_df['description']:  # Iterate directly over the description column
    embeddings = text_embedding_model.get_embeddings([description]) 
    text_embedding = [embedding.values for embedding in embeddings][0]
    embeddingsList.append(text_embedding)

shots_df["embedding"] = embeddingsList

In [9]:
shots_df.head()

,id,start_time,end_time,clip_name,frame_name,description,embedding
0,be_careful_bk.mp4_1,0,2.34,be_careful_bk.mp4,be_careful_bk.mp4_1.jpg,be_careful_bk,"[0.034322116523981094, 0.01670219749212265, 0...."
1,sailor_bk.mp4_1,0,2.90,sailor_bk.mp4,sailor_bk.mp4_1.jpg,sailor_bk,"[0.009677188470959663, 0.02842838689684868, 0...."
2,should.mp4_1,0,4.74,should.mp4,should.mp4_1.jpg,should,"[0.06107574701309204, 0.005806656088680029, -0..."
3,today.mp4_1,0,2.16,today.mp4,today.mp4_1.jpg,today,"[-0.005217599682509899, 0.023500807583332062, ..."
4,afternoon.mp4_1,0,1.84,afternoon.mp4,afternoon.mp4_1.jpg,afternoon,"[-0.001268067047931254, -0.030368715524673462,..."


In [10]:
# JSONL Export
jsonl_string = shots_df[['id','start_time','end_time','clip_name', 'frame_name', "description", "embedding"]].to_json(orient="records", lines=True)
with open(f"./videodata.json", "w") as f:
    f.write(jsonl_string)

In [11]:
! gsutil cp videodata.json {BUCKET_URI_ME}

Copying file://videodata.json [Content-Type=application/json]...
/ [1 files][101.9 KiB/101.9 KiB]                                                
Operation completed over 1 objects/101.9 KiB.                                    


### Creating Vector Search Index

In [12]:
# create Index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"vs-feature-index-{UID}",
    contents_delta_uri=BUCKET_URI_ME,
    dimensions=768,
    approximate_neighbors_count=10,
    project = PROJECT_ID
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/255766800726/locations/us-central1/indexes/2219337832386789376/operations/5222191121060331520
MatchingEngineIndex created. Resource name: projects/255766800726/locations/us-central1/indexes/2219337832386789376
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/255766800726/locations/us-central1/indexes/2219337832386789376')


### Create Index Endpoint and deploy the Index
To use the Index, you need to create an Index Endpoint. It works as a server instance accepting query requests for your Index.


In [15]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"sing-lang-testing-lh2-{UID}", public_endpoint_enabled=True
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/255766800726/locations/us-central1/indexEndpoints/5811943697633247232/operations/4935368119792173056
MatchingEngineIndexEndpoint created. Resource name: projects/255766800726/locations/us-central1/indexEndpoints/5811943697633247232
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/255766800726/locations/us-central1/indexEndpoints/5811943697633247232')


In [ ]:
DEPLOYED_INDEX_ID = f"sing_lang_testing_lh2_{UID}"
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID)

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/255766800726/locations/us-central1/indexEndpoints/5811943697633247232
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/255766800726/locations/us-central1/indexEndpoints/5811943697633247232/operations/274142505463709696
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/255766800726/locations/us-central1/indexEndpoints/5811943697633247232


resource name: projects/255766800726/locations/us-central1/indexEndpoints/5811943697633247232

### get index

In [ ]:
%pip install tqdm -q
import tqdm
import time


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import Image as vision_model_Image
from vertexai.vision_models import MultiModalEmbeddingModel
REGION = LOCATION = "us-central1"

aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
my_index_endpoint_id= '9221168615552712704' ##'2987475519047467008' #'8368299436119425024'
PROJECT_ID

'my-project-0004-346516'

In [ ]:
my_index_name = my_index._gca_resource.name
my_index_display_name = my_index.display_name
my_index_id = my_index.name.split('/')[-1]

my_index_endpoint_name = my_index_endpoint._gca_resource.name
my_index_endpoint_display_name = my_index_endpoint.display_name
my_index_endpoint_id = my_index_endpoint.name.split('/')[-1]
my_index_endpoint_public_domain = my_index_endpoint.public_endpoint_domain_name

my_index = aiplatform.MatchingEngineIndex(my_index_name)

my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(my_index_endpoint_id)

In [ ]:
from google.cloud import aiplatform_v1

# Set variables for the current deployed index.
API_ENDPOINT=my_index_endpoint_public_domain
INDEX_ENDPOINT=my_index_endpoint_name

indexendpoint_id=UNIQUE_PREFIX

DEPLOYED_INDEX_ID="sing_lang_testing_lh2_" + indexendpoint_id
neighbor_count = 3

print(API_ENDPOINT)
print(INDEX_ENDPOINT)
print(DEPLOYED_INDEX_ID)

94718965.us-central1-255766800726.vdb.vertexai.goog
projects/255766800726/locations/us-central1/indexEndpoints/5811943697633247232
sing_lang_testing_lh2_pytorch112kagglewbi


In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model="text-embedding-004", model_name="text-embedding-004")

text_embedding_model = embeddings #TextEmbeddingModel.from_pretrained("textembedding-gecko@003")

In [ ]:
#Set your query
query = "rain"

#Set the number of results you want
neighbor_count = 2

test_embeddings = embeddings.embed_query(query)

In [ ]:
# Configure Vector Search client
client_options = {
  "api_endpoint": API_ENDPOINT
}
vector_search_client = aiplatform_v1.MatchServiceClient(
  client_options=client_options,
)
# Build FindNeighborsRequest object
datapoint = aiplatform_v1.IndexDatapoint(
  feature_vector=test_embeddings
)

query = aiplatform_v1.FindNeighborsRequest.Query(
  datapoint=datapoint,
  # The number of nearest neighbors to be retrieved
  neighbor_count=neighbor_count
)

request = aiplatform_v1.FindNeighborsRequest(
  index_endpoint=INDEX_ENDPOINT,
  deployed_index_id=DEPLOYED_INDEX_ID,
  # Request can have multiple queries
  queries=[query],
  return_full_datapoint=False,
)

# Execute the request
response = vector_search_client.find_neighbors(request)

df_new = pd.DataFrame()
print('neighbor_count', neighbor_count)

shots_df['distance'] = None

for i in range(0,neighbor_count):
    x=response.nearest_neighbors[0]

    df_match = shots_df.loc[shots_df['id'] == str(x.neighbors[i].datapoint.datapoint_id) ]
    df_match['distance'] = x.neighbors[i].distance

    # Append the matching rows to the new DataFrame
    df_new = pd.concat([df_new, df_match])
    

# Print the new DataFrame
df_sorted = df_new.sort_values(by="distance", ascending=True)
print(display(df_sorted))

#Export DataFrame to CSV file for reference
# df_new.to_csv("results.csv")

neighbor_count 3


,id,start_time,end_time,clip_name,frame_name,description,embedding,distance
3,today.mp4_1,0,2.16,today.mp4,today.mp4_1.jpg,today,"[-0.005217599682509899, 0.023500807583332062, ...",0.412280
4,afternoon.mp4_1,0,1.84,afternoon.mp4,afternoon.mp4_1.jpg,afternoon,"[-0.001268067047931254, -0.030368715524673462,...",0.515925
7,pouring_rain.mp4_1,0,3.30,pouring_rain.mp4,pouring_rain.mp4_1.jpg,pouring_rain,"[0.00017337207100354135, -0.05368410795927048,...",0.843044


None


In [ ]:
# clipNames = []

# for i in df_sorted['clip_name']:
#     clipNames.append(i)
    
# #Display the clips     
# print(clipNames)

['today.mp4', 'afternoon.mp4', 'pouring_rain.mp4']


In [ ]:
# import vertexai

# from vertexai.generative_models import GenerativeModel, Part

# # TODO(developer): Update project_id and location
# vertexai.init(project=PROJECT_ID, location=LOCATION)

# model = GenerativeModel("gemini-1.5-flash-001")

# prompt = f"""
# Given the following query:
# Query: {query}
# Search Results: {clipNames}

# Select the best matching reult and only give out the final search result as an answer.
# """

# response = model.generate_content(
#     [ prompt]
# )

# print(response.text)

pouring_rain.mp4 



### Searching the full sentence 

In [ ]:
vertexai.init(project=PROJECT_ID, location=LOCATION)

model = GenerativeModel("gemini-1.5-pro-001")

query = "today pouring rain in thailand"
query = "sailor should be careful"

neighbor_count = 2


In [29]:
# Split the query into individual words
query_words = query.split()
stop_words = ["are", "of", "the", "an", "in", "an", "being", "a", "so", "an"]
filtered_words = [word for word in query_words if word.lower() not in stop_words]

video_list = []

for word in filtered_words:
    df_new = pd.DataFrame()
    
    test_embeddings = embeddings.embed_query(word)

    client_options = {
        "api_endpoint": API_ENDPOINT
    }
    vector_search_client = aiplatform_v1.MatchServiceClient(
        client_options=client_options,
    )

    datapoint = aiplatform_v1.IndexDatapoint(
        feature_vector=test_embeddings
    )

    query_embeded = aiplatform_v1.FindNeighborsRequest.Query(
        datapoint=datapoint,
        neighbor_count=neighbor_count
    )

    request = aiplatform_v1.FindNeighborsRequest(
        index_endpoint=INDEX_ENDPOINT,
        deployed_index_id=DEPLOYED_INDEX_ID,
        queries=[query_embeded],
        return_full_datapoint=False,
    )

    response = vector_search_client.find_neighbors(request)

    for i in range(0, neighbor_count):
        x = response.nearest_neighbors[0]
        df_match = shots_df.loc[shots_df['id'] == str(x.neighbors[i].datapoint.datapoint_id)]
        df_match['distance'] = x.neighbors[i].distance
        df_new = pd.concat([df_new, df_match])

    df_sorted = df_new.sort_values(by="distance", ascending=True)
    # print(display(df_sorted))

    clipNames = []

    for i in df_sorted['clip_name']:
        clipNames.append(i)

    print("word",word,clipNames)
    
    prompt = f"""
    Given the following query:
    Query: {query_words}
    Search Results: {clipNames}

    Select the best matching word with the action mp4 file and only give out the final search result as an answer, else give NA
    
    follow the logig of few_shot_examples : - 
    hello : hello.mp4
    welcome : welcome.mp4
    paris : paris.mp4
    forecast : forecast.mp4
    by : NA
    in : NA
    of : NA
    
    DO NOT give Anything else just the final match.
    
    """
    
    # print('prompt',prompt)
    response = model.generate_content([prompt])

    print(response.text)
    video_list.append(str(response.text))

word sailor ['sailor_v1.mp4', 'sailor.mp4']
sailor.mp4 

word should ['be_careful.mp4', 'should.mp4']
be_careful.mp4 

word be ['be_careful.mp4', 'be_careful_bk.mp4']
be_careful.mp4 

word careful ['be_careful_bk.mp4', 'be_careful.mp4']
be_careful.mp4 



In [30]:
# video_list
from itertools import groupby

cleaned_video_list = [item.strip() for item in video_list]
unique_video_list = [k for k, _ in groupby(cleaned_video_list)]

print(unique_video_list) 

final_video_list = unique_video_list

['sailor.mp4', 'be_careful.mp4']


In [31]:
import os
import subprocess

output_folder = './videos/'
output_file = 'concatenated_video_new.mp4'
output_file = 'sailorsvideo_new.mp4'

# Create a temporary text file to list the input videos
temp_file = 'input_list.txt'
with open(temp_file, 'w') as f:
    for video in final_video_list:
        f.write(f"file '{os.path.join(output_folder, video)}'\n")

# Construct the ffmpeg command
ffmpeg_command = [
    'ffmpeg',
    '-f', 'concat',
    '-safe', '0',
    '-i', temp_file,
    '-c', 'copy',  # Stream copy for fast concatenation
    '-y',    
    os.path.join('./', output_file)
]

# Execute the ffmpeg command
subprocess.run(ffmpeg_command)

# Clean up the temporary file
os.remove(temp_file)

print(f"Videos concatenated successfully into {output_file}")

ffmpeg version 4.3.7-0+deb11u1 Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 10 (Debian 10.2.1-6)
  configuration: --prefix=/usr --extra-version=0+deb11u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab 

Videos concatenated successfully into sailorsvideo_new.mp4


In [32]:
# output_file = 'concatenated_video_new.mp4'
# output_file = 'sailorsvideo_new.mp4'

from IPython.display import Video

Video(output_file)

In [ ]:
# from IPython.display import Video

# Video('./videos/be_careful.mp4')

In [ ]:
# # !ffmpeg -i ./videos/sailor.mp4 -c:v libx264 -crf 23 -c:a aac -b:a 128k ./videos/sailor.mp4 -y

# !ffmpeg -i ./videos/sailor.mp4 -r 30 ./videos/sailor.mp4
# !ffmpeg -i ./videos/be_careful.mp4 -r 30 ./videos/be_careful.mp4